# Measurements Kata Workbook, Part 2

The [Measurements Kata Workbook, Part 1](./Workbook_Measurements.ipynb) is still in progress and will include the solutions of kata Part I. This workbook continues the explanations for the rest of the tasks in Part II.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.10.2002.2610

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part II*. Discriminating Nonorthogonal States

Variations of quantum state discrimination tasks are covered in the paper ["Quantum State Discrimination"](https://arxiv.org/pdf/quant-ph/0010114.pdf).
* Task 2.1 is an example of hypothesis testing for two pure states.
* Task 2.2 is an example of unambiguous state discrimination. See also the paper ["Unambiguous quantum measurement of nonorthogonal states"](https://www.researchgate.net/publication/13375059_Unambiguous_quantum_measurement_of_nonorthogonal_states)
  for further information and hints about how to implement the unambiguous measurements required for this task. 
* Task 2.3 is the so-called Wootters/Peres game. See the following three references for more information and in particular
  the book [3, p. 287] for a nice description of the optimal POVM. 

  [1] A. Holevo, “Information-theoretical aspects of quantum measurement,” Problems of Information Transmission, vol. 9, no. 2, pp. 110–118 (1973)

  [2] A. Peres and W. K. Wootters, “Optimal detection of quantum information,” Phys. Rev. Lett., vol. 66, pp. 1119-1122, Mar. 1991.

  [3] A. Peres, “Quantum Theory: Concepts and Methods,” Kluwer Academic Publishers, 2002.

### <a name="task-21"></a>Task 2.1*. $|0\rangle$ or $|+\rangle$?

**Input:** A qubit which is guaranteed to be in either the $|0\rangle$ or the $|+\rangle$ state.

**Output:**  `true` if the qubit was in the $|0\rangle$ state, or `false` if it was in the $|+\rangle$ state. The state of the qubit at the end of the operation does not matter.

In this task your solution will be called multiple times, with one of the states picked with equal probability every time. You have to get overall accuracy of at least 80%.

This task is an example of quantum hypothesis testing, or state discrimination with minimum error.

#### Solution

Let ${|E_a⟩, |E_b⟩}$ be a measurement with two outcomes $a$ and $b$, which we identify with the answers, i.e., "a" = "state was |0⟩" and "b = state was |+⟩". Then we define
<li>
<ul>$P(a|0)$ = probability to observe first outcome given that the state was $|0⟩$</ul>
<ul>$P(b|0)$ = probability to observe second outcome given that the state was $|0⟩$</ul>
<ul>$P(a|+)$ = probability to observe first outcome given that the state was $|+⟩$</ul>
<ul>$P(b|+)$ = probability to observe second outcome given that the state was $|+⟩$</ul>
</li>
The task is to maximize the probability to be correct on a single shot experiment, which is the same as to minimize the probability to be wrong (on a single shot). Assuming uniform prior, i.e., $P(+) = P(0) = \frac{1}{2}$, we get<br>
$P_{correct} = P(0) P(a|0) + P(+) P(b|+) = \frac{1}{2} (P(a|0) + P(b|+))$

Assuming a von Neumann measurement of the form
<li>
<ul>$E_a = R_y(2\alpha) \begin{bmatrix} 1\\0\end{bmatrix} = \begin{bmatrix}\cos(\alpha) \\ \sin(\alpha)\end{bmatrix}$ and</ul>
<ul>$E_b = R_y(2\alpha) \begin{bmatrix} 0\\1\end{bmatrix} = \begin{bmatrix}-\sin(\alpha) \\ \cos(\alpha)\end{bmatrix}$, we get</ul>
<ul>$P(a|0) = |⟨E_a|0⟩|² = \cos^2(\alpha)$,</ul>
<ul>$P(b|+) = |⟨E_b|+⟩|² = \frac{1}{2} - \cos(\alpha) \sin(\alpha)$, and</ul>
<ul>$P_{correct} = \frac{1}{2} (\frac{1}{2} + \cos^2(\alpha) - \cos(\alpha) \sin(\alpha))$.</ul>
</li>
Maximizing this for $\alpha$, we get max $P_{success} = \frac{1}{2} (1 + \frac{1}{\sqrt{2}}) = 0.8535...$, which is attained for alpha = $-π/8$.
This means, $|E_a⟩$ is rotated by $-π/8$ compared to $0⟩$(analog $|E_b⟩$ and $1$). If we rotate the system by $-\alpha=π/8$ we will get $|E_a⟩=|0⟩$ and $|E_b⟩=|1⟩$ and a measurement in the computational basis will get the correct result with a probability of 85%.

![Task201_Rotation.png](./img/Task201_Rotation.png)

Note: Rotate the input state by $π/8$ means to apply $Ry$ with angle $2π/8$.
 

In [2]:
%kata T201_IsQubitZeroOrPlus

operation IsQubitPlusOrZero (q : Qubit) : Bool {
    
    // Rotate the input state by π/8 means to apply Ry with angle 2π/8.
    Ry(0.25 * PI(), q);
    return M(q) == Zero;
}

C:/snippet_.qs(1,85): error QS3001: Syntax does not match any known patterns.
C:/snippet_.qs(3,1): error QS6307: Not all code paths return a value.


[Return to task 2.1 of the Measurements kata.](./Measurements.ipynb#task-21)

### <a name="task-22"></a>Task 2.2**. $|0\rangle$, $|+\rangle$ or inconclusive?

**Input:** A qubit which is guaranteed to be in either the $|0\rangle$ or the $|+\rangle$ state.

**Output:** 

* 0 if the qubit was in the $|0\rangle$ state, 
* 1 if it was in the $|+\rangle$ state,
* -1 if you can't decide, i.e., an "inconclusive" result. 

Your solution:

* should never give 0 or 1 answer incorrectly (i.e., identify $|0\rangle$ as 1 or $|+\rangle$ as 0),
* will be called multiple times, with one of the states picked with equal probability every time,
* may give an inconclusive (-1) answer in at most 80% of all the cases,
* must correctly identify the $|0\rangle$ state as 0 in at least 10% of all the cases,
* must correctly identify the $|1\rangle$ state as 1 in at least 10% of all the cases.

The state of the qubit at the end of the operation does not matter.

This task is an example of unambiguous state discrimination.


#### Solution
A simple strategy that gives an inconclusive result with probability 0.75 and never errs in case it yields a conclusive result can be obtained from randomizing the choice of measurement basis between the computational basis (std) and the Hadamard basis (had). <br>
Observe that when measured in the standard basis, the state $|0⟩$ will always lead to the outcome "0", whereas the state $|+⟩$ will lead to outcomes "0" respectively "1" with probability 1/2. This means that upon measuring "1" we can with certainty conclude that the state was $|+⟩$. <br>
A similar argument applies to the scenario where we measure in the Hadamard basis, where $|0⟩$ can lead to both outcomes, whereas $|+⟩$ always leads to "0". <br>
Then upon measuring "1" we can with certainty conclude that the state was $|0⟩$.<br>
<br>
This leads to the following scenarios (shown are the conditional probabilities
of the above scenarios and resulting answers).


state | basis | output 0 | output 1 | output -1
------|-------|----------|----------|----------
  |0⟩ |   std |     0    |     0    |     1
  |+⟩ |   std |     0    |    1/2   |    1/2
  |0⟩ |   had |    1/2   |     0    |    1/2
  |+⟩ |   had |     0    |     0    |     1


In [3]:
%kata T202_IsQubitZeroOrPlusSimpleUSD

operation IsQubitPlusZeroOrInconclusiveSimpleUSD (q : Qubit) : Int {

    let basis = RandomInt(2);

    // randomize over std and had
    if (basis == 0) {
        
        // use standard basis
        let result = M(q);
        // result is One only if the state was |+⟩
        return result == One ? 1 | -1;
    }
    else {
        // use Hadamard basis
        H(q);
        let result = M(q);
        // result is One only if the state was |0⟩
        return result == One ? 0 | -1;
    }
}

C:/snippet_.qs(1,85): error QS3001: Syntax does not match any known patterns.
C:/snippet_.qs(5,17): error QS5022: No identifier with the name "RandomInt" exists.


[Return to task 2.2 of the Measurements kata.](./Measurements.ipynb#task-22)

### <a name="peter-wooters-game"></a>Task 2.3**.  Peres/Wooters game

**Input:** A qubit which is guaranteed to be in one of the three states:

* $|A\rangle = \frac{1}{\sqrt{2}} \big( |0\rangle + |1\rangle \big)$,
* $|B\rangle = \frac{1}{\sqrt{2}} \big( |0\rangle + \omega |1\rangle \big)$,
* $|C\rangle = \frac{1}{\sqrt{2}} \big( |0\rangle + \omega^2 |1\rangle \big)$,

Here $\omega = e^{2i \pi/ 3}$.

**Output:** 

* 1 or 2 if the qubit was in the $|A\rangle$ state, 
* 0 or 2 if the qubit was in the $|B\rangle$ state, 
* 0 or 1 if the qubit was in the $|C\rangle$ state.

You are never allowed to give an incorrect answer. Your solution will be called multiple times, with one of the states picked with equal probability every time.

The state of the qubit at the end of the operation does not matter. 

#### Solution
The key is the observation that the POVM with rank one elements ${E_0, E_1, E_2}$ will do the job, where $E_k = |psi_k\rangle\langle psi_k|$ and $|psi_k\rangle = 1/sqrt{2}(|0\rangle-ω^k |1\rangle)$ and where $k = 0, 1, 2$.<br> 
The remaining task will be to find a von Neumann measurement (on a qubit system) that implements said POVM by means of a quantum circuit. To obtain such a quantum circuit, we observe that all we need is a unitary extension of the matrix formed by $A = (psi_0, psi_1, psi_2)$. In general, such a unitary extension can be found using a Singular Value Decomposition of the matrix.<br>
Here we can apply a short cut:<br>
The matrix A can be extended (up to a +1/-1 diagonal) to a 3x3 Discrete Fourier Transform.<br>
Padded with 1 extra dimension (i.e., a 1x1 block in which we have phase freedom), we obtain a 4x4 unitary. Using the "Rader trick" we can now block decompose the 3x3 DFT and obtain two 2x2 blocks which we can then implement using controlled single qubit gates. We present the final resulting circuit without additional commentary.


In [ ]:
%kata T203_IsQubitNotInABC

operation IsQubitNotInABC (q : Qubit) : Int {
        
    let alpha = ArcCos(Sqrt(2.0 / 3.0));

    using (a = Qubit()) {
        Z(q);
        CNOT(a, q);
        Controlled H([q], a);
        S(a);
        X(q);

        (ControlledOnInt(0, Ry))([a], (-2.0 * alpha, q));
        CNOT(a, q);
        Controlled H([q], a);
        CNOT(a, q);

        // finally, measure in the standard basis
        let res0 = MResetZ(a);
        let res1 = M(q);

        // dispatch on the cases
        if (res0 == Zero and res1 == Zero) {
            return 0;
        }
        elif (res0 == One and res1 == Zero) {
            return 1;
        }
        elif (res0 == Zero and res1 == One) {
            return 2;
        }
        else {
            // this should never occur
            return 3;
        }
    }
}

[Return to task 2.3 of the Measurements kata.](./Measurements.ipynb#peter-wooters-game)